In [ ]:
# -*- coding: utf-8 -*-
"""
JackOFAllTrades: AI-Driven Trading System with Dynamic TP/SL Adjustments, Order Flow Analysis,
Risk Management, Spoofing Detection, and Closed Trade Logging.
"""

import MetaTrader5 as mt5
import pandas as pd
import numpy as np
import time
from datetime import datetime
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
import xgboost as xgb
import matplotlib.pyplot as plt

# --- Initialization ---
try:
    if not mt5.initialize():
        raise Exception("Failed to initialize MetaTrader 5.")
except Exception as e:
    print(f"Initialization error: {e}")
    quit()

# Account details
account_id = 5031478194  # Replace with your MT5 account ID
password = "*6WjKeRu"  # Replace with your MT5 account password
server = "MetaQuotes-Demo"

try:
    if not mt5.login(account_id, password=password, server=server):
        raise Exception("Login failed.")
    print("Connected to MetaTrader 5.")
except Exception as e:
    print(f"Login error: {e}")
    quit()

symbol = "XAUUSD"  # Replace with your desired trading symbol

# --- Helper Functions ---
def get_tick_data(symbol):
    """Fetch current tick data including bid and ask prices."""
    tick = mt5.symbol_info_tick(symbol)
    if tick is None:
        raise ValueError(f"Could not fetch tick data for symbol {symbol}")
    return tick

def fetch_market_data(symbol, num_bars=5):
    """Retrieve a smaller set of historical market data (5 bars)."""
    rates = mt5.copy_rates_from_pos(symbol, mt5.TIMEFRAME_M1, 0, num_bars)
    if rates is None or len(rates) == 0:
        print("Failed to fetch market data.")
        return None

    df = pd.DataFrame(rates)
    df['time'] = pd.to_datetime(df['time'], unit='s')
    df['volatility'] = df['high'] - df['low']

    # Add current tick data
    tick = get_tick_data(symbol)
    df['ask'] = tick.ask
    df['bid'] = tick.bid
    df['spread'] = df['ask'] - df['bid']

    df['target'] = (df['close'].shift(-1) > df['close']).astype(int)
    return df.dropna()

def prepare_ai_features(data):
    """Prepare features and target for AI model training."""
    features = data[["close", "volatility", "spread"]]
    target = data["target"]
    scaler = MinMaxScaler()
    features_scaled = scaler.fit_transform(features)
    return features_scaled, target

# --- XGBoost Model ---
def create_xgb_model():
    """Build and return the XGBoost model."""
    model = xgb.XGBClassifier(objective='binary:logistic', eval_metric='logloss')
    return model

# --- Order Flow Analysis ---
def analyze_order_flow(symbol):
    """Analyze order flow to determine the amount of buy/sell orders at price points."""
    try:
        market_book = mt5.market_book_get(symbol)
        
        if market_book is None:
            print(f"Market depth data not available for symbol: {symbol}")
            return None

        buy_orders, sell_orders = {}, {}
        total_buy_volume, total_sell_volume = 0, 0

        for entry in market_book:
            price, volume = entry.price, entry.volume
            if entry.type == mt5.BOOK_TYPE_BUY:
                total_buy_volume += volume
                buy_orders[price] = buy_orders.get(price, 0) + volume
            elif entry.type == mt5.BOOK_TYPE_SELL:
                total_sell_volume += volume
                sell_orders[price] = sell_orders.get(price, 0) + volume

        control = "buyers" if total_buy_volume > total_sell_volume else "sellers"
        dominance = abs(total_buy_volume - total_sell_volume) / (total_buy_volume + total_sell_volume) * 100

        print(f"Order flow control: {control} ({dominance:.2f}% dominance)")
        return {"control": control, "dominance": dominance}
    
    except Exception as e:
        print(f"Error in analyze_order_flow: {e}")
        return None
    
# --- Dynamic SL/TP Adjustment ---
def adjust_stop_loss_take_profit(model, symbol):
    """Dynamically adjust SL/TP for open positions."""
    try:
        positions = mt5.positions_get(symbol=symbol)
        if positions is None or len(positions) == 0:
            print(f"No open positions for symbol {symbol}.")
            return

        # Fetch market data and prepare features
        market_data = fetch_market_data(symbol)
        if market_data is None:
            print("Failed to fetch market data.")
            return
        features = prepare_ai_features(market_data)
        predicted_volatility = model.predict(features[-1:])[0]  # Predict using the last feature row

        # Loop through all positions
        for position in positions:
            volume = position.volume
            price_open = position.price_open
            sl = position.sl
            tp = position.tp
            trade_type = position.type  # 0 = Buy, 1 = Sell

            tick_size = mt5.symbol_info(symbol).point
            high = market_data['high'].iloc[-1]
            low = market_data['low'].iloc[-1]
            current_price = get_tick_data(symbol).bid if trade_type == 1 else get_tick_data(symbol).ask

            # Calculate new SL based on volatility
            if trade_type == mt5.ORDER_TYPE_BUY:
                new_sl = max(price_open, low - predicted_volatility - 10 * tick_size)
                if current_price > price_open + 20 * tick_size:  # Move SL to break-even if profitable
                    new_sl = max(new_sl, price_open)
                new_tp = current_price + predicted_volatility
            else:  # Sell trade
                new_sl = min(price_open, high + predicted_volatility + 10 * tick_size)
                if current_price < price_open - 20 * tick_size:  # Move SL to break-even if profitable
                    new_sl = min(new_sl, price_open)
                new_tp = current_price - predicted_volatility

            # Update the trade
            request = {
                "action": mt5.TRADE_ACTION_SLTP,
                "symbol": symbol,
                "sl": round(new_sl, 5),
                "tp": round(new_tp, 5),
                "position": position.ticket,
            }
            result = mt5.order_send(request)
            if result.retcode == mt5.TRADE_RETCODE_DONE:
                print(f"SL/TP updated for position {position.ticket}: SL={new_sl}, TP={new_tp}")
            else:
                print(f"Failed to update SL/TP for position {position.ticket}. Error: {result.retcode}")
    except Exception as e:
        print(f"Error adjusting SL/TP: {e}")
# --- Trade Execution ---
def place_trade(symbol, order_type, price, lot, sl_pips, tp_pips):
    """Place a trade with the given parameters."""
    try:
        tick_size = mt5.symbol_info(symbol).point
        sl = price - sl_pips * tick_size if order_type == mt5.ORDER_TYPE_BUY else price + sl_pips * tick_size
        tp = price + tp_pips * tick_size if order_type == mt5.ORDER_TYPE_BUY else price - tp_pips * tick_size

        request = {
            "action": mt5.TRADE_ACTION_DEAL,
            "symbol": symbol,
            "volume": lot,
            "type": order_type,
            "price": price,
            "sl": round(sl, 5),
            "tp": round(tp, 5),
            "deviation": 20,
            "magic": 234000,
            "comment": "Optimized AI Trade",
            "type_time": mt5.ORDER_TIME_GTC,
            "type_filling": mt5.ORDER_FILLING_RETURN,
        }
        result = mt5.order_send(request)
        if result.retcode == mt5.TRADE_RETCODE_DONE:
            print(f"Trade placed: {result}")
        else:
            print(f"Trade failed. Retcode: {result.retcode}")
    except Exception as e:
        print(f"Error placing trade: {e}")

# --- Main ---
if __name__ == "__main__":
    # Fetch market data and prepare AI features
    market_data = fetch_market_data(symbol)
    if market_data is None:
        mt5.shutdown()
        quit()

    features, target = prepare_ai_features(market_data)
    target = market_data['volatility']

    # Train the XGBoost model
    model = create_xgb_model()
    X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.1, shuffle=False)
    model.fit(X_train, y_train)

    # Evaluate model performance
    accuracy = model.score(X_test, y_test)
    print(f"Model Accuracy: {accuracy:.2f}")

    # Run trading loop
    while True:
        # Example trading logic here
        #Dynamic SL/TP adjustment Loop
        adjust_stop_loss_take_profit(model, symbol)
        time.sleep(1)  # Run every 5 seconds for demo

    # Shutdown MetaTrader 5
    mt5.shutdown()


In [ ]:
# -*- coding: utf-8 -*-
"""
Project Underdogg: Trading System with Post-Learning Cycle Trade Placement
Integrates AI Model Retraining, Accuracy Evaluation, and Dynamic TP/SL.
"""

import MetaTrader5 as mt5
import pandas as pd
import numpy as np
import time
from datetime import datetime
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
import xgboost as xgb

# --- Initialization ---
try:
    if not mt5.initialize():
        raise Exception("Failed to initialize MetaTrader 5.")
except Exception as e:
    print(f"Initialization error: {e}")
    quit()

# Account details
account_id = 5031478194  # Replace with your MT5 account ID
password = "*6WjKeRu"  # Replace with your MT5 account password
server = "MetaQuotes-Demo"

try:
    if not mt5.login(account_id, password=password, server=server):
        raise Exception("Login failed.")
    print("Connected to MetaTrader 5.")
except Exception as e:
    print(f"Login error: {e}")
    quit()

symbol = "XAUUSD"  # Replace with your desired trading symbol

# --- Helper Functions ---
def get_bid_price(symbol):
    """Fetch the current bid price for the given symbol."""
    tick = mt5.symbol_info_tick(symbol)
    return tick.bid if tick else None

def get_ask_price(symbol):
    """Fetch the current ask price for the given symbol."""
    tick = mt5.symbol_info_tick(symbol)
    return tick.ask if tick else None

def fetch_market_data(symbol, num_bars=15):
    """Retrieve historical market data with calculated spread."""
    rates = mt5.copy_rates_from_pos(symbol, mt5.TIMEFRAME_M1, 0, num_bars)
    if rates is None or len(rates) == 0:
        print("Failed to fetch market data.")
        return None

    df = pd.DataFrame(rates)
    df['time'] = pd.to_datetime(df['time'], unit='s')
    df['volatility'] = df['high'] - df['low']

    # Dynamically fetch the current spread using symbol_info_tick
    tick = mt5.symbol_info_tick(symbol)
    if tick:
        df['ask'] = tick.ask
        df['bid'] = tick.bid
        df['spread'] = tick.ask - tick.bid
    else:
        # Fallback values if tick data is unavailable
        df['ask'] = df['close']
        df['bid'] = df['close']
        df['spread'] = 0

    df['target'] = (df['close'].shift(-1) > df['close']).astype(int)
    return df.dropna()

def prepare_ai_features(data):
    """Prepare features and target for AI model training."""
    features = data[["close", "volatility", "spread"]]
    target = data["target"]
    scaler = MinMaxScaler()
    features_scaled = scaler.fit_transform(features)
    return features_scaled, target

# --- XGBoost Model ---
def create_xgb_model():
    """Build and return the XGBoost model without deprecated parameters."""
    model = xgb.XGBClassifier(objective='binary:logistic', eval_metric='logloss')
    return model
def adjust_stop_loss_take_profit(model, symbol):
    """Dynamically adjust SL/TP for open positions."""
    try:
        positions = mt5.positions_get(symbol=symbol)
        if positions is None or len(positions) == 0:
            print(f"No open positions for symbol {symbol}.")
            return

        # Fetch market data and prepare features
        market_data = fetch_market_data(symbol)
        if market_data is None:
            print("Failed to fetch market data.")
            return
        features = prepare_ai_features(market_data)
        predicted_volatility = model.predict(features[-1:])[0]  # Predict using the last feature row

        # Loop through all positions
        for position in positions:
            volume = position.volume
            price_open = position.price_open
            sl = position.sl
            tp = position.tp
            trade_type = position.type  # 0 = Buy, 1 = Sell

            tick_size = mt5.symbol_info(symbol).point
            high = market_data['high'].iloc[-1]
            low = market_data['low'].iloc[-1]
            current_price = get_tick_data(symbol).bid if trade_type == 1 else get_tick_data(symbol).ask

            # Calculate new SL based on volatility
            if trade_type == mt5.ORDER_TYPE_BUY:
                new_sl = max(price_open, low - predicted_volatility - 10 * tick_size)
                if current_price > price_open + 20 * tick_size:  # Move SL to break-even if profitable
                    new_sl = max(new_sl, price_open)
                new_tp = current_price + predicted_volatility
            else:  # Sell trade
                new_sl = min(price_open, high + predicted_volatility + 10 * tick_size)
                if current_price < price_open - 20 * tick_size:  # Move SL to break-even if profitable
                    new_sl = min(new_sl, price_open)
                new_tp = current_price - predicted_volatility

            # Update the trade
            request = {
                "action": mt5.TRADE_ACTION_SLTP,
                "symbol": symbol,
                "sl": round(new_sl, 5),
                "tp": round(new_tp, 5),
                "position": position.ticket,
            }
            result = mt5.order_send(request)
            if result.retcode == mt5.TRADE_RETCODE_DONE:
                print(f"SL/TP updated for position {position.ticket}: SL={new_sl}, TP={new_tp}")
            else:
                print(f"Failed to update SL/TP for position {position.ticket}. Error: {result.retcode}")
    except Exception as e:
        print(f"Error adjusting SL/TP: {e}")


# --- Trade Execution ---
def place_trade(symbol, order_type, price, lot, sl_pips, tp_pips):
    """Place a trade with the given parameters."""
    try:
        tick_size = mt5.symbol_info(symbol).point
        sl = price - sl_pips * tick_size if order_type == mt5.ORDER_TYPE_BUY else price + sl_pips * tick_size
        tp = price + tp_pips * tick_size if order_type == mt5.ORDER_TYPE_BUY else price - tp_pips * tick_size

        request = {
            "action": mt5.TRADE_ACTION_DEAL,
            "symbol": symbol,
            "volume": lot,
            "type": order_type,
            "price": price,
            "sl": round(sl, 5),
            "tp": round(tp, 5),
            "deviation": 20,
            "magic": 234000,
            "comment": "Post-Learning AI Trade",
            "type_time": mt5.ORDER_TIME_GTC,
            "type_filling": mt5.ORDER_FILLING_RETURN,
        }
        result = mt5.order_send(request)
        if result.retcode == mt5.TRADE_RETCODE_DONE:
            print(f"Trade placed: {result}")
        else:
            print(f"Trade failed. Retcode: {result.retcode}")
    except Exception as e:
        print(f"Error placing trade: {e}")

# --- Trading Logic ---
def enhanced_trading_logic(symbol, model):
    """Executes trades after retraining the AI model."""
    try:
        # Fetch market data
        market_data = fetch_market_data(symbol)
        if market_data is None:
            return

        # Prepare features and make predictions
        features, target = prepare_ai_features(market_data)
        predictions = model.predict(features)

        # Decision-making and trade execution
        for i, pred in enumerate(predictions):
            if pred == 1:  # Predicted upward movement
                place_trade(symbol, mt5.ORDER_TYPE_BUY, market_data.iloc[i]['ask'], 0.1, 20, 40)
            elif pred == 0:  # Predicted downward movement
                place_trade(symbol, mt5.ORDER_TYPE_SELL, market_data.iloc[i]['bid'], 0.1, 20, 40)

    except Exception as e:
        print(f"Error in trading logic: {e}")

# --- Main ---
if __name__ == "__main__":
    # Fetch initial market data
    market_data = fetch_market_data(symbol)
    if market_data is None:
        mt5.shutdown()
        quit()

    # Prepare features and target
    features, target = prepare_ai_features(market_data)
    target = market_data['volatility']

    # Initialize the XGBoost model
    model = create_xgb_model()

    while True:
        try:
            # Retrain the model during each cycle
            X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.1, shuffle=False)
            model.fit(X_train, y_train)

            # Evaluate model accuracy
            accuracy = model.score(X_test, y_test)
            print(f"Model Accuracy: {accuracy:.2f}")

            # If accuracy is acceptable, place trades
            if accuracy > 0.7:  # Use a threshold to ensure good performance
                print("Model accuracy is high. Executing trades...")
                enhanced_trading_logic(symbol, model)
            else:
                print("Model accuracy is too low. Skipping trade placement.")

            # Wait before the next cycle
            adjust_stop_loss_profit(model, symbol)
            time.sleep(5)

        except Exception as e:
            print(f"Error in main loop: {e}")
            time.sleep(5)

    # Shutdown MetaTrader 5
    mt5.shutdown()



In [ ]:
# -*- coding: utf-8 -*-
"""
Trading System with AI Model and Order Flow Analysis
"""

import MetaTrader5 as mt5
import pandas as pd
import numpy as np
import time
from datetime import datetime
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
import xgboost as xgb

# --- Initialization ---
try:
    if not mt5.initialize():
        raise Exception("Failed to initialize MetaTrader 5.")
except Exception as e:
    print(f"Initialization error: {e}")
    quit()

# Account details
account_id = 5031478194  # Replace with your MT5 account ID
password = "*6WjKeRu"  # Replace with your MT5 account password
server = "MetaQuotes-Demo"

try:
    if not mt5.login(account_id, password=password, server=server):
        raise Exception("Login failed.")
    print("Connected to MetaTrader 5.")
except Exception as e:
    print(f"Login error: {e}")
    quit()

symbol = "XAUUSD"  # Replace with your desired trading symbol

# --- Helper Functions ---
def get_bid_price(symbol):
    """Fetch the current bid price for the given symbol."""
    tick = mt5.symbol_info_tick(symbol)
    return tick.bid if tick else None

def get_ask_price(symbol):
    """Fetch the current ask price for the given symbol."""
    tick = mt5.symbol_info_tick(symbol)
    return tick.ask if tick else None

def fetch_market_data(symbol, num_bars=15):
    """Retrieve historical market data with calculated spread."""
    rates = mt5.copy_rates_from_pos(symbol, mt5.TIMEFRAME_M1, 0, num_bars)
    if rates is None or len(rates) == 0:
        print("Failed to fetch market data.")
        return None

    df = pd.DataFrame(rates)
    df['time'] = pd.to_datetime(df['time'], unit='s')
    df['volatility'] = df['high'] - df['low']

    # Dynamically fetch the current spread using symbol_info_tick
    tick = mt5.symbol_info_tick(symbol)
    if tick:
        df['ask'] = tick.ask
        df['bid'] = tick.bid
        df['spread'] = tick.ask - tick.bid
    else:
        df['ask'] = df['close']
        df['bid'] = df['close']
        df['spread'] = 0

    df['target'] = (df['close'].shift(-1) > df['close']).astype(int)
    return df.dropna()

def prepare_ai_features(data):
    """Prepare features and target for AI model training."""
    features = data[["close", "volatility", "spread"]]
    target = data["target"]
    scaler = MinMaxScaler()
    features_scaled = scaler.fit_transform(features)
    return features_scaled, target

# --- Order Flow Analysis ---
def fetch_order_book(symbol):
    """Fetch order book data for the given symbol."""
    book = mt5.market_book_get(symbol)
    if book is None:
        print(f"No order book data for {symbol}.")
        return None
    return pd.DataFrame(book)

def calculate_vwap(order_book):
    """Calculate VWAP from order book data."""
    if order_book is None or order_book.empty:
        return None
    order_book['value'] = order_book['price'] * order_book['volume']
    total_volume = order_book['volume'].sum()
    vwap = order_book['value'].sum() / total_volume if total_volume > 0 else None
    return vwap

def modify_sl_tp_with_partial_closure(symbol, partial_target_profit, lot_fraction=0.5):
    """
    Adjust SL/TP dynamically with partial profit-taking and breakeven logic.

    Args:
        symbol (str): Trading symbol (e.g., XAUUSD).
        partial_target_profit (float): Profit target for partial closure.
        lot_fraction (float): Fraction of the position to close during partial closure.
    """
    try:
        positions = mt5.positions_get(symbol=symbol)
        if positions is None or len(positions) == 0:
            print(f"No open positions for symbol {symbol}.")
            return

        for position in positions:
            trade_type = position.type  # 0 = Buy, 1 = Sell
            current_price = get_ask_price(symbol) if trade_type == mt5.ORDER_TYPE_BUY else get_bid_price(symbol)

            if current_price is None:
                print(f"Failed to fetch current price for {symbol}. Skipping adjustment.")
                continue

            # Check if SL is already at breakeven or beyond
            if (trade_type == mt5.ORDER_TYPE_BUY and position.sl >= position.price_open) or \
               (trade_type == mt5.ORDER_TYPE_SELL and position.sl <= position.price_open):
                print(f"Position {position.ticket}: SL already at breakeven or beyond. Skipping adjustment.")
                continue

            # Check if price has reached the partial profit target
            if (trade_type == mt5.ORDER_TYPE_BUY and current_price >= partial_target_profit) or \
               (trade_type == mt5.ORDER_TYPE_SELL and current_price <= partial_target_profit):
                
                print(f"Partial profit target reached for position {position.ticket}. Initiating partial closure...")

                # Perform partial closure
                lot_to_close = round(position.volume * lot_fraction, 2)  # Round to valid lot size
                request_close = {
                    "action": mt5.TRADE_ACTION_DEAL,
                    "symbol": symbol,
                    "volume": lot_to_close,
                    "type": mt5.ORDER_TYPE_SELL if trade_type == mt5.ORDER_TYPE_BUY else mt5.ORDER_TYPE_BUY,
                    "position": position.ticket,
                    "price": current_price,
                    "deviation": 50,
                    "magic": position.magic,
                    "comment": "Partial closure",
                    "type_time": mt5.ORDER_TIME_GTC,
                    "type_filling": mt5.ORDER_FILLING_RETURN,
                }

                result_close = mt5.order_send(request_close)
                if result_close.retcode == mt5.TRADE_RETCODE_DONE:
                    print(f"Partial closure successful for position {position.ticket}: Closed {lot_to_close} lots.")
                else:
                    print(f"Failed to partially close position {position.ticket}. Error: {result_close.retcode}")
                    continue

                # Move stop loss to breakeven for remaining position
                new_sl = position.price_open
                request_sl_update = {
                    "action": mt5.TRADE_ACTION_SLTP,
                    "symbol": symbol,
                    "position": position.ticket,
                    "sl": round(new_sl, 5),
                    "tp": position.tp,  # Keep the same TP
                }

                result_sl_update = mt5.order_send(request_sl_update)
                if result_sl_update.retcode == mt5.TRADE_RETCODE_DONE:
                    print(f"Moved SL to breakeven for position {position.ticket}. SL={new_sl}")
                else:
                    print(f"Failed to move SL to breakeven for position {position.ticket}. Error: {result_sl_update.retcode}")

    except Exception as e:
        print(f"Error in modifying SL/TP with partial closure: {e}")

def analyze_imbalances(order_book):
    """Detect bid/ask imbalances from the order book."""
    if order_book is None or order_book.empty:
        return None
    bid_volume = order_book.loc[order_book['type'] == mt5.BOOK_TYPE_BID, 'volume'].sum()
    ask_volume = order_book.loc[order_book['type'] == mt5.BOOK_TYPE_ASK, 'volume'].sum()
    imbalance = bid_volume - ask_volume
    return imbalance

# --- XGBoost Model ---
def create_xgb_model():
    """Build and return the XGBoost model without deprecated parameters."""
    model = xgb.XGBClassifier(objective='binary:logistic', eval_metric='logloss')
    return model

# --- Trade Execution ---
def place_trade(symbol, order_type, price, lot, sl_pips, tp_pips):
    """Place a trade with the given parameters."""
    try:
        tick_size = mt5.symbol_info(symbol).point
        sl = price - sl_pips * tick_size if order_type == mt5.ORDER_TYPE_BUY else price + sl_pips * tick_size
        tp = price + tp_pips * tick_size if order_type == mt5.ORDER_TYPE_BUY else price - tp_pips * tick_size

        request = {
            "action": mt5.TRADE_ACTION_DEAL,
            "symbol": symbol,
            "volume": lot,
            "type": order_type,
            "price": price,
            "sl": round(sl, 5),
            "tp": round(tp, 5),
            "deviation": 20,
            "magic": 234000,
            "comment": "Jack Of All Trades",
            "type_time": mt5.ORDER_TIME_GTC,
            "type_filling": mt5.ORDER_FILLING_RETURN,
        }
        result = mt5.order_send(request)
        if result.retcode == mt5.TRADE_RETCODE_DONE:
            print(f"Trade placed: {result}")
        else:
            print(f"Trade failed. Retcode: {result.retcode}")
    except Exception as e:
        print(f"Error placing trade: {e}")

# --- Main Loop ---
# --- Main Loop ---
if __name__ == "__main__":
    # Fetch initial market data
    market_data = fetch_market_data(symbol)
    if market_data is None:
        print("Could not retrieve market data")
        mt5.shutdown()
        quit()

    # Prepare features and target
    features, target = prepare_ai_features(market_data)

    # Initialize the XGBoost model
    model = create_xgb_model()

    while True:
        try:
            # Retrain the model during each cycle
            X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.1, shuffle=False)
            model.fit(X_train, y_train)

            # Evaluate model accuracy
            accuracy = model.score(X_test, y_test)
            print(f"Model Accuracy: {accuracy:.2f}")

            # Fetch order book and calculate VWAP/imbalances
            order_book = fetch_order_book(symbol)
            vwap = calculate_vwap(order_book)
            imbalance = analyze_imbalances(order_book)
            print(f"VWAP: {vwap}, Imbalance: {imbalance}")

            # Place trades based on model predictions
            if accuracy > 0.7:
                print("Model accuracy is high. Executing trades...")
                enhanced_trading_logic(symbol, model)
                
                # Adjust SL/TP dynamically with partial closure
                #partial_target = 50  # Example target profit in points
               # modify_sl_tp_with_partial_closure(symbol, partial_target_profit=partial_target)
            else:
                print("Model accuracy is too low. Skipping trade placement.")

            time.sleep(60)  # Adjust execution interval

        except Exception as e:
            print(f"Error in main loop: {e}")



In [ ]:
# -*- coding: utf-8 -*-
"""
Ultimate Trading Bot: AI Model, Order Flow, Dynamic SL/TP Adjustments, and Risk Management
"""

import MetaTrader5 as mt5
import pandas as pd
import numpy as np
import time
from datetime import datetime
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
import xgboost as xgb

# --- Initialization ---
try:
    if not mt5.initialize():
        raise Exception("Failed to initialize MetaTrader 5.")
except Exception as e:
    print(f"Initialization error: {e}")
    quit()

# Account details
account_id = 5031478194  # Replace with your MT5 account ID
password = "*6WjKeRu"  # Replace with your MT5 account password
server = "MetaQuotes-Demo"

try:
    if not mt5.login(account_id, password=password, server=server):
        raise Exception("Login failed.")
    print("Connected to MetaTrader 5.")
except Exception as e:
    print(f"Login error: {e}")
    quit()

symbol = "XAUUSD"  # Replace with your desired trading symbol

# --- Helper Functions ---
def get_bid_price(symbol):
    """Fetch the current bid price for the given symbol."""
    tick = mt5.symbol_info_tick(symbol)
    return tick.bid if tick else None

def get_ask_price(symbol):
    """Fetch the current ask price for the given symbol."""
    tick = mt5.symbol_info_tick(symbol)
    return tick.ask if tick else None

def fetch_market_data(symbol, num_bars=15):
    """Retrieve historical market data."""
    rates = mt5.copy_rates_from_pos(symbol, mt5.TIMEFRAME_M1, 0, num_bars)
    if rates is None or len(rates) == 0:
        print("Failed to fetch market data.")
        return None

    df = pd.DataFrame(rates)
    df['time'] = pd.to_datetime(df['time'], unit='s')
    return df

def prepare_ai_features(data):
    """Prepare features and target for AI model training."""
    features = data[["close", "high", "low", "tick_volume"]]
    target = (data["close"].shift(-1) > data["close"]).astype(int)
    scaler = MinMaxScaler()
    features_scaled = scaler.fit_transform(features)
    return features_scaled, target

# --- Dynamic SL/TP Adjustments ---
def calculate_dynamic_sl_tp(trade_type, high, low, tick_size):
    """Calculate dynamic SL and TP based on previous candle high and low."""
    pip_adjustment = 10 * tick_size
    if trade_type == mt5.ORDER_TYPE_BUY:
        sl = low - pip_adjustment
        tp = high + pip_adjustment
    elif trade_type == mt5.ORDER_TYPE_SELL:
        sl = high + pip_adjustment
        tp = low - pip_adjustment
    return round(sl, 5), round(tp, 5)

def adjust_stop_loss_take_profit(symbol):
    """Dynamically adjust SL/TP for open positions."""
    positions = mt5.positions_get(symbol=symbol)
    if positions is None or len(positions) == 0:
        print(f"No open positions for symbol {symbol}.")
        return

    market_data = fetch_market_data(symbol)
    if market_data is None:
        return

    high = market_data['high'].iloc[-1]
    low = market_data['low'].iloc[-1]
    tick_size = mt5.symbol_info(symbol).point

    for position in positions:
        trade_type = position.type
        current_price = get_ask_price(symbol) if trade_type == mt5.ORDER_TYPE_BUY else get_bid_price(symbol)
        new_sl, new_tp = calculate_dynamic_sl_tp(trade_type, high, low, tick_size)

        if trade_type == mt5.ORDER_TYPE_BUY and current_price > position.price_open:
            new_sl = max(new_sl, position.price_open)
        elif trade_type == mt5.ORDER_TYPE_SELL and current_price < position.price_open:
            new_sl = min(new_sl, position.price_open)

        request = {
            "action": mt5.TRADE_ACTION_SLTP,
            "symbol": symbol,
            "position": position.ticket,
            "sl": new_sl,
            "tp": new_tp,
        }
        result = mt5.order_send(request)
        if result.retcode == mt5.TRADE_RETCODE_DONE:
            print(f"Updated SL/TP for position {position.ticket}: SL={new_sl}, TP={new_tp}")
        else:
            print(f"Failed to update SL/TP for position {position.ticket}. Error: {result.retcode}")

# --- Order Flow Analysis ---
def fetch_order_book(symbol):
    """Fetch order book data for the given symbol."""
    book = mt5.market_book_get(symbol)
    if book is None:
        print(f"No order book data for {symbol}.")
        return None
    return pd.DataFrame(book)

def calculate_vwap(order_book):
    """Calculate VWAP from order book data."""
    if order_book is None or order_book.empty:
        return None
    order_book['value'] = order_book['price'] * order_book['volume']
    total_volume = order_book['volume'].sum()
    vwap = order_book['value'].sum() / total_volume if total_volume > 0 else None
    return vwap

def analyze_imbalances(order_book):
    """Detect bid/ask imbalances from the order book."""
    if order_book is None or order_book.empty:
        return None
    bid_volume = order_book.loc[order_book['type'] == mt5.BOOK_TYPE_BID, 'volume'].sum()
    ask_volume = order_book.loc[order_book['type'] == mt5.BOOK_TYPE_ASK, 'volume'].sum()
    imbalance = bid_volume - ask_volume
    return imbalance

# --- XGBoost Model ---
def create_xgb_model():
    """Build and return the XGBoost model."""
    model = xgb.XGBClassifier(objective='binary:logistic', eval_metric='logloss')
    return model

# --- Trade Execution ---
def place_trade(symbol, order_type, price, lot, sl, tp):
    """Place a trade with the given parameters."""
    request = {
        "action": mt5.TRADE_ACTION_DEAL,
        "symbol": symbol,
        "volume": lot,
        "type": order_type,
        "price": price,
        "sl": sl,
        "tp": tp,
        "deviation": 20,
        "magic": 234000,
        "comment": "Ultimate Trading Bot",
        "type_time": mt5.ORDER_TIME_GTC,
        "type_filling": mt5.ORDER_FILLING_RETURN,
    }
    result = mt5.order_send(request)
    if result.retcode == mt5.TRADE_RETCODE_DONE:
        print(f"Trade placed: {result}")
    else:
        print(f"Trade failed. Retcode: {result.retcode}")

# --- Main Loop ---
if __name__ == "__main__":
    market_data = fetch_market_data(symbol)
    if market_data is None:
        mt5.shutdown()
        quit()

    features, target = prepare_ai_features(market_data)
    model = create_xgb_model()

    while True:
        try:
            X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.1, shuffle=False)
            model.fit(X_train, y_train)
            accuracy = model.score(X_test, y_test)
            print(f"Model Accuracy: {accuracy:.2f}")

            if accuracy > 0.7:
                print("Model accuracy is high. Executing trades...")
                adjust_stop_loss_take_profit(symbol)

            time.sleep(60)
        except Exception as e:
            print(f"Error in main loop: {e}")
            time.sleep(60)

    mt5.shutdown()


In [ ]:
# -*- coding: utf-8 -*-
"""
Ultimate Trading System with AI, Order Flow, Risk Management, Dynamic Adjustments,
Spoofing Detection, and Trade Logging.
"""

import MetaTrader5 as mt5
import pandas as pd
import numpy as np
import time
from datetime import datetime
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from statsmodels.tsa.arima.model import ARIMA
from sklearn.cluster import KMeans

# --- Initialization ---
try:
    if not mt5.initialize():
        raise Exception("Failed to initialize MetaTrader 5.")
except Exception as e:
    print(f"Initialization error: {e}")
    quit()

# Account details
account_id = 5031478194  # Replace with your MT5 account ID
password = "*6WjKeRu"  # Replace with your MT5 account password
server = "MetaQuotes-Demo"

try:
    if not mt5.login(account_id, password=password, server=server):
        raise Exception("Login failed.")
    print("Connected to MetaTrader 5.")
except Exception as e:
    print(f"Login error: {e}")
    quit()

symbol = "XAUUSD"  # Replace with your desired trading symbol

# --- Helper Functions ---
def get_bid_price(symbol):
    """Fetch the current bid price for the given symbol."""
    tick = mt5.symbol_info_tick(symbol)
    return tick.bid if tick else None

def get_ask_price(symbol):
    """Fetch the current ask price for the given symbol."""
    tick = mt5.symbol_info_tick(symbol)
    return tick.ask if tick else None

def fetch_market_data(symbol, num_bars=5):
    """Retrieve historical market data with calculated spread."""
    rates = mt5.copy_rates_from_pos(symbol, mt5.TIMEFRAME_M1, 0, num_bars)
    if rates is None or len(rates) == 0:
        print("Failed to fetch market data.")
        return None

    df = pd.DataFrame(rates)
    df['time'] = pd.to_datetime(df['time'], unit='s')
    df['volatility'] = df['high'] - df['low']

    # Dynamically fetch the current spread using symbol_info_tick
    tick = mt5.symbol_info_tick(symbol)
    if tick:
        df['ask'] = tick.ask
        df['bid'] = tick.bid
        df['spread'] = tick.ask - tick.bid
    else:
        df['ask'] = df['close']
        df['bid'] = df['close']
        df['spread'] = 0

    df['target'] = (df['close'].shift(-1) > df['close']).astype(int)
    return df.dropna()

# --- Order Flow Analysis ---
def analyze_order_flow(symbol, market_data):
    """Analyze order flow using market book data."""
    market_book = mt5.market_book_get(symbol)
    if market_book is None:
        print("Order flow data unavailable.")
        return None

    # Convert the market book to a DataFrame
    order_book_df = pd.DataFrame(market_book)

    # Calculate the total buy and sell volumes
    buy_volume = order_book_df[order_book_df['type'] == mt5.BOOK_TYPE_BUY]['volume'].sum()
    sell_volume = order_book_df[order_book_df['type'] == mt5.BOOK_TYPE_SELL]['volume'].sum()

    # Calculate the imbalance (difference between buy and sell volumes)
    imbalance = buy_volume - sell_volume

    # Output the imbalance value
    print(f"Order Flow Imbalance: {imbalance}")

    # Check if there is more accumulation of buy orders than sell orders
    if buy_volume > sell_volume:
        return 1.0

    # Return the buy and sell volumes along with the imbalance
    elif sell_volume > buy_volume:
        return -1.0
    
    else:
        return -11
    


# --- Trend Analysis ---
def identify_trend(market_data):
    """Identify market trend using ARIMA."""
    prices = market_data['close']
    model = ARIMA(prices, order=(1, 1, 0))
    model_fit = model.fit()

    trend_signal = model_fit.forecast(steps=1)[0]
    print(f"Trend Signal: {trend_signal}")
    return trend_signal

# --- Dynamic Adjustments ---
def adjust_sl_tp(order_flow, trend):
    """Dynamically adjust SL and TP levels."""
    market_data = fetch_market_data(symbol)
    last_closing_price = market_data['close'].iloc[-1]
    #uptrend
    if (trend > last_closing_price):
        sl = trend - 50  # SL logic
        tp = trend + 50  # TP logic
    #downtrend
    elif (trend < last_closing_price):
        sl = trend + 50
        tp = trend - 50
    print(f"Adjusted SL: {sl}, TP: {tp}")
    return {"sl": sl, "tp": tp}

#SL for open
def adjust_sl_for_all_trades(symbol, sl_buffer_pips=10):
    """
    Adjust the Stop Loss (SL) for all trades dynamically based on their profit status.

    Args:
        symbol (str): The trading symbol (e.g., "XAUUSD").
        sl_buffer_pips (int): The buffer (in pips) to place the SL for profitable trades.

    Returns:
        None
    """
    try:
        positions = mt5.positions_get(symbol=symbol)
        if positions is None or len(positions) == 0:
            print(f"No open positions for {symbol}.")
            return

        for position in positions:
            trade_type = position.type  # 0 = BUY, 1 = SELL
            current_price = get_ask_price(symbol) if trade_type == mt5.ORDER_TYPE_BUY else get_bid_price(symbol)
            tick_size = mt5.symbol_info(symbol).point
            sl_buffer = sl_buffer_pips * tick_size

            # Check if the trade is in profit
            if (trade_type == mt5.ORDER_TYPE_BUY and current_price > position.price_open) or \
               (trade_type == mt5.ORDER_TYPE_SELL and current_price < position.price_open):
                print(f"Position {position.ticket} is in profit. Adjusting SL.")

                # Calculate new SL for profitable trades
                new_sl = current_price - sl_buffer if trade_type == mt5.ORDER_TYPE_BUY else current_price + sl_buffer

                # Ensure SL doesn't move backward
                if (trade_type == mt5.ORDER_TYPE_BUY and new_sl > position.sl) or \
                   (trade_type == mt5.ORDER_TYPE_SELL and new_sl < position.sl):
                    # Send SL modification request
                    request = {
                        "action": mt5.TRADE_ACTION_SLTP,
                        "symbol": symbol,
                        "position": position.ticket,
                        "sl": round(new_sl, 5),
                        "tp": position.tp  # Keep the current TP unchanged
                    }
                    result = mt5.order_send(request)

                    if result.retcode == mt5.TRADE_RETCODE_DONE:
                        print(f"SL updated for profitable position {position.ticket}: New SL = {new_sl}")
                    else:
                        print(f"Failed to update SL for position {position.ticket}. Error: {result.retcode}")
                else:
                    print(f"New SL does not improve protection for position {position.ticket}. No change made.")

            else:
                print(f"Position {position.ticket} is not in profit. Adjusting SL to opening price.")

                # Adjust SL to the opening price for non-profitable trades
                new_sl = position.price_open
                request = {
                    "action": mt5.TRADE_ACTION_SLTP,
                    "symbol": symbol,
                    "position": position.ticket,
                    "sl": round(new_sl, 5),
                    "tp": position.tp  # Keep the current TP unchanged
                }
                result = mt5.order_send(request)

                if result.retcode == mt5.TRADE_RETCODE_DONE:
                    print(f"SL updated to opening price for position {position.ticket}: New SL = {new_sl}")
                else:
                    print(f"Failed to update SL for position {position.ticket}. Error: {result.retcode}")

    except Exception as e:
        print(f"Error adjusting SL for all trades: {e}")



def scale_lot_size(market_data):
    """Dynamically scale lot size based on market conditions."""
    volatility = market_data['volatility'].iloc[-1]
    lot_size = max(0.1, 1 / volatility)  # Example scaling
    print(f"Lot Size: {lot_size}")
    return lot_size

# --- Risk Management ---
def manage_risk(lot_size, max_margin=0.7):
    """Ensure margin usage stays within limits."""
    account_info = mt5.account_info()
    if account_info is None:
        print("Account info unavailable.")
        return False

    margin_used = account_info.margin_used / account_info.balance
    print(f"Margin Used: {margin_used}")
    return margin_used < max_margin

# --- AI Model ---
def train_ai_model(features, target):
    """Train AI model on market data."""
    #features = market_data[["close", "volatility", "spread"]]
    #target = market_data["target"]
    model = RandomForestClassifier()
    #model.fit(features)
    model.fit(features, target)
    return model

def evaluate_model(model, features, target):
    """Evaluate model accuracy."""
    accuracy = model.score(features, target)
    print(f"Model Accuracy: {accuracy}")
    return accuracy

# --- Spoofing Detection ---
def detect_spoofing(order_flow):
    """Detect spoofing using clustering."""
    data = [[order_flow['buy_volume']], [order_flow['sell_volume']]]
    kmeans = KMeans(n_clusters=2)
    kmeans.fit(data)
    labels = kmeans.labels_

    if len(set(labels)) == 1:  # Potential spoofing
        print("Spoofing detected!")

# --- Trade Execution ---
def place_trade(symbol, order_type, price, lot, sl_pips, tp_pips):
    """Place a trade with the given parameters."""
    try:
        tick_size = mt5.symbol_info(symbol).point
        sl = price - sl_pips * tick_size if order_type == mt5.ORDER_TYPE_BUY else price + sl_pips * tick_size
        tp = price + tp_pips * tick_size if order_type == mt5.ORDER_TYPE_BUY else price - tp_pips * tick_size

        request = {
            "action": mt5.TRADE_ACTION_DEAL,
            "symbol": symbol,
            "volume": lot,
            "type": order_type,
            "price": price,
            "sl": round(sl, 5),
            "tp": round(tp, 5),
            "deviation": 20,
            "magic": 234000,
            "comment": "Jack Of All Trades",
            "type_time": mt5.ORDER_TIME_GTC,
            "type_filling": mt5.ORDER_FILLING_RETURN,
        }
        result = mt5.order_send(request)
        if result.retcode == mt5.TRADE_RETCODE_DONE:
            print(f"Trade placed: {result}")
        else:
            print(f"Trade failed. Retcode: {result.retcode}")
    except Exception as e:
        print(f"Error placing trade: {e}")
        
def prepare_ai_features(data):
    """Prepare features and target for AI model training."""
    features = data[["close", "high", "low", "tick_volume"]]
    target = (data["close"].shift(-1) > data["close"]).astype(int)
    scaler = MinMaxScaler()
    features_scaled = scaler.fit_transform(features)
    return features_scaled, target

# --- Trade Logging ---
def log_trade_data(symbol):
    """Log trade data for further analysis."""
    positions = mt5.positions_get(symbol=symbol)
    if positions is None:
        print("No trades to log.")
        return

    trade_data = [{"symbol": pos.symbol, "volume": pos.volume, "price": pos.price_open} for pos in positions]
    trade_df = pd.DataFrame(trade_data)
    trade_df.to_csv("trade_log.csv", index=False)
    print("Trade data logged.")

# --- Main Loop ---
def main():
    # Fetch initial market data
    market_data = fetch_market_data(symbol)
    if market_data is None:
        mt5.shutdown()
        quit()

   

    

    while True:
        
        try:
            
             # Prepare features and target
            features, target = prepare_ai_features(market_data)
            print("Preparing AI features and target")                                      
            
            # Initialize the Random Forest model
            model = train_ai_model(features, target)
            print("Starting Random walk")
            # Evaluate model accuracy
            accuracy = evaluate_model(model, features, target)
            print("Evaluating Model Accuracy")

            # Fetch order book and analyze
            order_book = analyze_order_flow(symbol, market_data)
            print("Fetching Order book")
            trend = identify_trend(market_data)
            print("Identifying market trend")

            
            lot_size = scale_lot_size(market_data)
            print("adjusting lot size")

            # Risk Management
            if not manage_risk(lot_size):                                                                                                                                                                                              
                print("Risk limits exceeded. Skipping trade placement.")
                continue

            # Place Trades
            last_closing_price = market_data['close'].iloc[-1]
            if (trend > last_closing_price) and (accuracy > 0.95) and (order_book == "Bull"):
                print("Uptrend detected: Entering BUY")
                price = get_ask_price(symbol)
                sl_tp = adjust_sl_tp(order_book, trend)
                print("Setting SL and TP for new trades")
                place_trade(symbol, mt5.ORDER_TYPE_BUY, price, lot_size, sl_tp['sl'], sl_tp['tp'])
                adjust_sl_for_all_trades(symbol)
                # Log Trades
                log_trade_data(symbol)
                # Spoofing Detection
                detect_spoofing(order_book)
            elif (trend < last_closing_price) and (accuracy > 0.95) and (order_book == "Bear"):
                print("Downtrend detected: Entering Sell")
                price = get_ask_price(symbol)
                sl_tp = adjust_sl_tp(order_book, trend)
                print("Setting SL and TP for new trades")
                place_trade(symbol, mt5.ORDER_TYPE_SELL, price, lot_size, sl_tp['sl'], sl_tp['tp'])
                adjust_sl_for_all_trades(symbol)
                print("Modified open trades sl and tp")
                # Log Trades
                log_trade_data(symbol)
                # Spoofing Detection
                detect_spoofing(order_book)
                
            else:
                Print("No change detected: No Trade detected")
                print(accuracy)
                adjust_sl_for_all_trades(symbol)
                print("Modified open trades sl and tp")
                # Log Trades
                log_trade_data(symbol)
                # Spoofing Detection
                detect_spoofing(order_book)
            


            # Sleep for the next iteration
            time.sleep(1)

        except Exception as e:
            print(f"Error in main loop: {e}")
            time.sleep(1)

if __name__ == "__main__":
    main()


Connected to MetaTrader 5.
Preparing AI features and target
Starting Random walk
Model Accuracy: 1.0
Evaluating Model Accuracy
Error in main loop: 'type'
Preparing AI features and target
Starting Random walk
Model Accuracy: 1.0
Evaluating Model Accuracy
Error in main loop: 'type'
Preparing AI features and target
Starting Random walk
Model Accuracy: 1.0
Evaluating Model Accuracy
Error in main loop: 'type'
Preparing AI features and target
Starting Random walk
Model Accuracy: 1.0
Evaluating Model Accuracy
Error in main loop: 'type'
Preparing AI features and target
Starting Random walk
Model Accuracy: 1.0
Evaluating Model Accuracy
Error in main loop: 'type'
Preparing AI features and target
Starting Random walk
Model Accuracy: 1.0
Evaluating Model Accuracy
Error in main loop: 'type'
Preparing AI features and target
Starting Random walk
Model Accuracy: 1.0
Evaluating Model Accuracy
Error in main loop: 'type'
Preparing AI features and target
Starting Random walk
Model Accuracy: 1.0
Evaluating